In [3]:
import pandas as pd
import jinja2
from IPython.display import display, Javascript, HTML

> You may need to adjust the ***height*** depending on your output.

In [4]:
def inline_dc(stuff):
    """
    Embeds the HTML source of the dc charts directly into the IPython notebook.
    
    This method will not work if the dc charts depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """

    return HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 300px; border: none"></iframe>'.format(srcdoc=stuff.replace('"', '&quot;')))

# Import JS libraries  

> These may need to be updated as they rely on third party links

In [5]:
head = HTML("""
<head> 
 
<link rel="stylesheet" type="text/css" href="https://cdnjs.cloudflare.com/ajax/libs/dc/2.0.0-alpha.2/dc.css"/> 
<script src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min.js"></script> 
<script src="https://cdnjs.cloudflare.com/ajax/libs/crossfilter/1.3.9/crossfilter.min.js"></script> 
<script src="https://cdnjs.cloudflare.com/ajax/libs/dc/1.7.0/dc.min.js"></script> 

</head>
""")

# Create Front-End

In [6]:
body_html = HTML("""


<!-- A div anchor that can be identified by id -->
<div id="piechart" class="dc-chart">
    <!-- Title or anything you want to add above the chart -->
    <strong>Pie Chart</strong>
    <!--
        This will create a reset button when ever the user selects a filter.
        Clicking on the reset button will remove all filters.
     -->
    <a class="reset" href="javascript:PieChart.filterAll();dc.redrawAll();" style="display: none;">reset</a>
    <div class="clearfix"></div>
    <!--
        dc.js will also automatically inject applied current filter value into
        any html element with css class set to "filter"
    -->
    <span class="reset" style="display: none;">Current filter: <span class="filter"></span></span>
    <br>
    </br>
</div>


""")

# Get Data

In [7]:
df = pd.DataFrame({'population':[234,345,345,34],
                   'age':[2,3,6,9],
                   'state':['a','b','c','d'],
                   'state2':['d','c','b','a']})
df

,age,population,state,state2
0,2,234,a,d
1,3,345,b,c
2,6,345,c,b
3,9,34,d,a


# Javascript Template

In [8]:
dc = jinja2.Template(
"""   

        // Create Global Variables
        var PieChart = dc.pieChart("#piechart");

        // Load data
        var dataset = {{ data }};

        // Call function
        Graph(dataset);


// Create function
function Graph(data) {

   
    // Feed it through crossfilter  
    var ndx = crossfilter(data);
    
    // for testing
    //console.log(data);

    //define a dimension
    //Here we will group by state
    var dim = ndx.dimension(function(d) {return d.state});

    //Here we group by state and sum on column population
    var g = dim.group().reduceSum(function(d){return d.population;});
    
    //Lets create a pie chart
    PieChart.dimension(dim)
            .radius(90)
            .innerRadius(45) // used to create the donut effect
            .group(g)

            // These last two lines are not needed but are
            // here to show you how to create custom titles
            .title(function(d){ return d.data.key +": "+d.value;})
            .renderTitle(true);     

    dc.renderAll(); // render all charts on the page
    
}; // end graph function    
    
""")



# Bind data to JS Template

In [9]:
body_js = Javascript(dc.render(
    data=df.to_json(orient='records')
    ))

# Push Data to the Notebook

In [10]:
inline_dc(head.data + body_html.data + "<script>" + body_js.data + "</script>")

**Author:** [David Rojas LLC](http://hdrojas.pythonanywhere.com/)  